In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
dataset = pd.read_csv(r"./Datasets/FTSE/FTSE_MFDFA.csv")
df = dataset.reset_index(drop=True)
df

,Unnamed: 0.1,Unnamed: 0,Date,Open,High,Low,Close,QPLp1,QPLp2,QPLp3,...,QPLn13,QPLn14,QPLn15,QPLn16,QPLn17,QPLn18,QPLn19,QPLn20,QPLn21,Hurst
0,255,255,2011/1/6,23861.22,23861.22,23698.38,23786.30,23918.438207,24052.945967,24220.047025,...,21362.217500,21129.343629,20896.134525,20663.016281,20430.360846,20198.493439,19967.698638,19738.225444,19510.291550,0.591327
1,256,256,2011/1/7,23744.03,23880.77,23654.04,23686.63,23800.967190,23934.814340,24101.094712,...,21257.300892,21025.570738,20793.506998,20561.533671,20330.020880,20099.292248,19869.630953,19641.284774,19414.470336,0.609555
2,257,257,2011/1/10,23715.47,23783.83,23481.80,23527.26,23772.338704,23906.024859,24072.105224,...,21231.732001,21000.280579,20768.495972,20536.801669,20305.567349,20075.116243,19845.731191,19617.659674,19391.118055,0.612508
3,258,258,2011/1/11,23606.76,23855.83,23512.22,23760.34,23663.368022,23796.441369,23961.760434,...,21134.407277,20904.016811,20673.294687,20442.662455,20212.488096,19983.093361,19754.759794,19527.733740,19302.230571,0.608676
4,259,259,2011/1/12,23857.17,24168.36,23857.17,24125.61,23914.378495,24048.863425,24215.936121,...,21358.591660,21125.757315,20892.587793,20659.509117,20426.893171,20195.065119,19964.309491,19734.875246,19506.980040,0.608713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,1968,1968,2017/12/19,29193.74,29339.11,29144.17,29253.66,29263.745366,29428.313003,29632.757908,...,26136.258897,25851.342232,25566.015415,25280.799763,24996.150349,24712.465073,24430.092109,24149.336106,23870.463406,0.476967
1714,1969,1969,2017/12/20,29199.29,29326.52,29159.28,29234.09,29269.308675,29433.907597,29638.391369,...,26141.227642,25856.256811,25570.875751,25285.605877,25000.902349,24717.163141,24434.736496,24153.927118,23875.001402,0.458090
1715,1970,1970,2017/12/21,29209.64,29431.81,29145.13,29367.06,29279.683494,29444.340760,29648.897013,...,26150.493679,25865.421837,25579.939620,25294.568630,25009.764185,24725.924403,24443.397649,24162.488735,23883.464151,0.438911
1716,1971,1971,2017/12/22,29505.05,29578.01,29413.86,29578.01,29575.801875,29742.124392,29948.749414,...,26414.965180,26127.010281,25838.640856,25550.383783,25262.698985,24975.988606,24690.604533,24406.854663,24125.008181,0.428980


In [2]:
maxLevel = 21                           
maxTP = 1                
maxTS = 4096             
nTP = 0                    
p3 = 1.0 / 3.0          

stime = 0
etime = 0
Gstime = 0
Getime = 0
tlapse = 0
Gtlapse = 0

day = 2261

DT_DT= np.zeros((day,1))

DT_OP= np.zeros((day,1))
DT_HI= np.zeros((day,1))
DT_LO= np.zeros((day,1))
DT_CL= np.zeros((day,1))
DT_VL= np.zeros((day,1))

cHurst= np.zeros((day,1))
oHurst= np.zeros((day,1))
hHurst= np.zeros((day,1))
lHurst= np.zeros((day,1))
DT_RT= np.zeros((day,1))

Q= np.zeros((100,1))
NQ= np.zeros((100,1))
r= np.zeros((100,1))


QFEL = np.zeros((21,1))       
QPR = np.zeros((21,1))       
NQPR = np.zeros((21,1))      
K = np.zeros((21,1))


ALL_Pos_QPL = np.zeros((day,21))
ALL_Neg_QPL = np.zeros((day,21))

print("Printout ALL K values K0 .. K20 for first 20 Levels")
for L in range(0,21):    
    K[L] = math.pow((1.1924 + (33.2383 * L) + (56.2169 * L * L)) / (1 + (43.6106 * L)), p3)
    print("Level ",L, " K",L, " = ", K[L])   

Printout ALL K values K0 .. K20 for first 20 Levels
Level  0  K 0  =  [1.06041043]
Level  1  K 1  =  [1.2665998]
Level  2  K 2  =  [1.4911995]
Level  3  K 3  =  [1.66349934]
Level  4  K 4  =  [1.80609901]
Level  5  K 5  =  [1.92919156]
Level  6  K 6  =  [2.03832293]
Level  7  K 7  =  [2.13688129]
Level  8  K 8  =  [2.22710524]
Level  9  K 9  =  [2.31055991]
Level  10  K 10  =  [2.38838747]
Level  11  K 11  =  [2.46144992]
Level  12  K 12  =  [2.53041565]
Level  13  K 13  =  [2.59581465]
Level  14  K 14  =  [2.65807505]
Level  15  K 15  =  [2.71754825]
Level  16  K 16  =  [2.77452666]
Level  17  K 17  =  [2.82925649]
Level  18  K 18  =  [2.88194724]
Level  19  K 19  =  [2.93277882]
Level  20  K 20  =  [2.98190702]


In [3]:
DT_DT = df['Date']     
DT_OP = df['Open']
DT_HI = df['High']
DT_LO = df['Low']
DT_CL = df['Close']
hurst = df['Hurst']
TSsize = df.shape[0]

TSsize

1718

In [4]:
# For Hurst
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_CL[d + 1] > 0):
        DT_RT[d] = DT_CL[d] / DT_CL[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurst[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurst[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/FTSE/FTSE_MFDFA+qhl.csv",index = True)